# OpenSearch Neural Sparse Model - 한국어 추론 테스트

학습된 모델을 사용하여 한국어 샘플에 대한 추론을 수행하고 토큰화 정보를 분석합니다.

## 목표
- 학습된 모델 로드 및 검증
- 한국어 쿼리/문서 샘플 추론
- 토큰화 정보 상세 분석
- Sparse representation 시각화

## 1. Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path
import json
from typing import List, Dict, Tuple

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

from src.model.splade_model import SPLADEDoc

print(f"Project root: {project_root}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. 학습된 모델 로드

In [ ]:
# 모델 경로 설정 (best_model 사용)
model_path = project_root / "outputs" / "baseline_dgx" / "best_model"
checkpoint_path = model_path / "checkpoint.pt"

print(f"Loading model from: {model_path}")
print(f"Checkpoint exists: {checkpoint_path.exists()}")

if not checkpoint_path.exists():
    print("\n⚠️ WARNING: Checkpoint not found!")
    print("Please check the model path or train the model first.")
    print(f"Expected location: {checkpoint_path}")
else:
    # 체크포인트 로드
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    
    print("\n📦 Checkpoint loaded!")
    print(f"  Epoch: {checkpoint['epoch']}")
    print(f"  Global step: {checkpoint['global_step']}")
    print(f"  Best validation loss: {checkpoint['best_val_loss']:.4f}")
    
    # 설정 정보
    config = checkpoint['config']
    model_name = config['model']['name']
    
    print(f"\n⚙️ Model Configuration:")
    print(f"  Base model: {model_name}")
    print(f"  Max length: {config['data'].get('max_length', 256)}")
    
    # SPLADEDoc 모델 초기화
    model = SPLADEDoc(
        model_name=model_name,
        dropout=config['model'].get('dropout', 0.1),
    )
    
    # 학습된 가중치 로드
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    
    # 모델 정보 저장
    vocab_size = model.config.vocab_size
    hidden_size = model.config.hidden_size
    max_length = config['data'].get('max_length', 256)
    
    print("\n✓ Model loaded successfully!")
    print(f"  Model name: {model_name}")
    print(f"  Vocab size: {vocab_size}")
    print(f"  Hidden size: {hidden_size}")
    print(f"  Max length: {max_length}")

## 3. Tokenizer 로드

In [ ]:
# Tokenizer 로드 (모델과 동일한 토크나이저 사용)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Tokenizer loaded")
print(f"  Model: {model_name}")
print(f"  Vocab size: {len(tokenizer)}")
print(f"  Model max length: {tokenizer.model_max_length}")

# 특수 토큰 확인
print("\nSpecial tokens:")
print(f"  PAD: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"  CLS: {tokenizer.cls_token} (ID: {tokenizer.cls_token_id})")
print(f"  SEP: {tokenizer.sep_token} (ID: {tokenizer.sep_token_id})")
print(f"  UNK: {tokenizer.unk_token} (ID: {tokenizer.unk_token_id})")

## 4. 한국어 샘플 데이터 준비

In [ ]:
# 한국어 샘플 쿼리
korean_queries = [
    "머신러닝이란 무엇인가",
    "딥러닝 모델 학습 방법",
    "자연어 처리 기술",
    "컴퓨터 비전 알고리즘",
    "데이터 분석 도구",
]

# 한국어 샘플 문서
korean_documents = [
    "머신러닝은 컴퓨터가 데이터로부터 학습하는 인공지능의 한 분야입니다.",
    "딥러닝은 인공 신경망을 사용하여 복잡한 패턴을 학습하는 머신러닝 기법입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 처리하는 기술입니다.",
    "컴퓨터 비전은 이미지와 비디오를 분석하여 의미 있는 정보를 추출하는 기술입니다.",
    "데이터 분석은 대량의 데이터를 수집, 처리, 분석하여 인사이트를 도출하는 과정입니다.",
]

print(f"Prepared {len(korean_queries)} queries and {len(korean_documents)} documents")
print("\nSample query:")
print(f"  {korean_queries[0]}")
print("\nSample document:")
print(f"  {korean_documents[0]}")

## 5. 토큰화 정보 분석 함수

In [ ]:
def analyze_tokenization(
    text: str,
    tokenizer: AutoTokenizer,
    max_length: int = 64,
) -> Dict:
    """
    텍스트의 토큰화 정보를 분석합니다.
    
    Args:
        text: 분석할 텍스트
        tokenizer: 토크나이저
        max_length: 최대 길이
        
    Returns:
        토큰화 정보 딕셔너리
    """
    # 토큰화
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'][0]
    attention_mask = encoding['attention_mask'][0]
    
    # 토큰 텍스트 변환
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    # 실제 토큰 (패딩 제외)
    actual_length = attention_mask.sum().item()
    actual_tokens = tokens[:actual_length]
    actual_ids = input_ids[:actual_length].tolist()
    
    return {
        'text': text,
        'input_ids': input_ids.tolist(),
        'attention_mask': attention_mask.tolist(),
        'tokens': tokens,
        'actual_tokens': actual_tokens,
        'actual_ids': actual_ids,
        'actual_length': actual_length,
        'total_length': len(tokens),
    }


def display_tokenization(token_info: Dict) -> None:
    """
    토큰화 정보를 보기 좋게 출력합니다.
    """
    print("="*80)
    print(f"Text: {token_info['text']}")
    print("="*80)
    print(f"\nLength: {token_info['actual_length']} tokens (padded to {token_info['total_length']})")
    print("\nTokens:")
    
    # 토큰 테이블 생성
    data = []
    for i, (token, token_id) in enumerate(zip(token_info['actual_tokens'], token_info['actual_ids'])):
        data.append({
            'Position': i,
            'Token': token,
            'ID': token_id,
        })
    
    df = pd.DataFrame(data)
    print(df.to_string(index=False))
    print()

print("Tokenization analysis functions defined")

## 6. 샘플 토큰화 테스트

In [ ]:
# 첫 번째 쿼리 토큰화 분석
sample_query = korean_queries[0]
query_token_info = analyze_tokenization(sample_query, tokenizer, max_length=64)
display_tokenization(query_token_info)

In [ ]:
# 첫 번째 문서 토큰화 분석
sample_doc = korean_documents[0]
doc_token_info = analyze_tokenization(sample_doc, tokenizer, max_length=256)
display_tokenization(doc_token_info)

## 7. 모델 추론 및 Sparse Representation 분석

In [ ]:
def encode_text(text: str, model: SPLADEDoc, tokenizer: AutoTokenizer) -> torch.Tensor:
    """
    텍스트를 sparse representation으로 인코딩합니다.
    
    Args:
        text: 입력 텍스트
        model: SPLADEDoc 모델
        tokenizer: 토크나이저
        
    Returns:
        Sparse representation [vocab_size]
    """
    # 토큰화
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # 모델 추론
    with torch.no_grad():
        sparse_rep, _ = model(input_ids, attention_mask)
    
    return sparse_rep[0]  # [vocab_size]


def analyze_sparse_representation(
    text: str,
    model: SPLADEDoc,
    tokenizer: AutoTokenizer,
    top_k: int = 20,
) -> Dict:
    """
    텍스트의 sparse representation을 분석합니다.
    
    Args:
        text: 분석할 텍스트
        model: SPLADEDoc 모델
        tokenizer: 토크나이저
        top_k: 상위 k개 활성화 토큰
        
    Returns:
        분석 결과 딕셔너리
    """
    # 토큰화 정보
    token_info = analyze_tokenization(text, tokenizer, max_length)
    
    # 모델 추론
    sparse_rep = encode_text(text, model, tokenizer)
    
    # Sparse representation 분석
    sparse_rep_np = sparse_rep.cpu().numpy()
    
    # 0이 아닌 값만 추출
    nonzero_indices = np.nonzero(sparse_rep_np)[0]
    nonzero_values = sparse_rep_np[nonzero_indices]
    
    # 상위 k개 추출
    if len(nonzero_values) > 0:
        top_k_actual = min(top_k, len(nonzero_values))
        top_k_indices = np.argsort(nonzero_values)[-top_k_actual:][::-1]
        top_k_token_ids = nonzero_indices[top_k_indices]
        top_k_values = nonzero_values[top_k_indices]
        top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_token_ids)
    else:
        top_k_token_ids = np.array([])
        top_k_values = np.array([])
        top_k_tokens = []
    
    # 통계
    stats = {
        'num_activated': len(nonzero_indices),
        'vocab_size': len(sparse_rep_np),
        'sparsity': 1.0 - (len(nonzero_indices) / len(sparse_rep_np)),
        'mean_activation': nonzero_values.mean() if len(nonzero_values) > 0 else 0.0,
        'max_activation': nonzero_values.max() if len(nonzero_values) > 0 else 0.0,
        'min_activation': nonzero_values.min() if len(nonzero_values) > 0 else 0.0,
    }
    
    return {
        'text': text,
        'token_info': token_info,
        'sparse_rep': sparse_rep_np,
        'nonzero_indices': nonzero_indices,
        'nonzero_values': nonzero_values,
        'top_k_token_ids': top_k_token_ids,
        'top_k_values': top_k_values,
        'top_k_tokens': top_k_tokens,
        'stats': stats,
    }


def display_sparse_analysis(analysis: Dict) -> None:
    """
    Sparse representation 분석 결과를 출력합니다.
    """
    print("="*80)
    print(f"Text: {analysis['text']}")
    print("="*80)
    
    stats = analysis['stats']
    print("\n📊 Sparsity Statistics:")
    print(f"  Vocab size: {stats['vocab_size']:,}")
    print(f"  Activated tokens: {stats['num_activated']:,}")
    print(f"  Sparsity: {stats['sparsity']:.2%}")
    print(f"  Mean activation: {stats['mean_activation']:.4f}")
    print(f"  Max activation: {stats['max_activation']:.4f}")
    print(f"  Min activation: {stats['min_activation']:.4f}")
    
    print(f"\n🔥 Top {len(analysis['top_k_tokens'])} Activated Terms:")
    
    # Top-k 테이블
    data = []
    for i, (token_id, token, value) in enumerate(
        zip(analysis['top_k_token_ids'], analysis['top_k_tokens'], analysis['top_k_values']),
        1
    ):
        data.append({
            'Rank': i,
            'Token': token,
            'ID': token_id,
            'Weight': f"{value:.4f}",
            'Bar': '█' * int(value * 20),
        })
    
    df = pd.DataFrame(data)
    print(df.to_string(index=False))
    print()

print("Sparse representation analysis functions defined")

## 8. 쿼리 Sparse Representation 분석

In [ ]:
# 첫 번째 쿼리 분석
query_analysis = analyze_sparse_representation(
    korean_queries[0],
    model,
    tokenizer,
    top_k=20,
)
display_sparse_analysis(query_analysis)

## 9. 문서 Sparse Representation 분석

In [ ]:
# 첫 번째 문서 분석
doc_analysis = analyze_sparse_representation(
    korean_documents[0],
    model,
    tokenizer,
    top_k=20,
)
display_sparse_analysis(doc_analysis)

## 10. 여러 샘플 비교 분석

In [ ]:
# 모든 쿼리 분석
print("="*80)
print("ALL QUERIES ANALYSIS")
print("="*80)

query_analyses = []
for i, query in enumerate(korean_queries, 1):
    print(f"\n[Query {i}/{len(korean_queries)}]")
    analysis = analyze_sparse_representation(query, model, tokenizer, top_k=10)
    query_analyses.append(analysis)
    
    # 간단한 요약만 출력
    stats = analysis['stats']
    print(f"Text: {query}")
    print(f"  Activated: {stats['num_activated']:,} tokens ({100-stats['sparsity']*100:.2f}%)")
    print(f"  Top 5 terms: {', '.join(analysis['top_k_tokens'][:5])}")

## 11. Sparsity 비교 시각화

In [ ]:
# Sparsity 비교 그래프
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Query sparsity
query_sparsities = [a['stats']['sparsity'] for a in query_analyses]
query_activated = [a['stats']['num_activated'] for a in query_analyses]

ax1 = axes[0]
x = range(1, len(query_sparsities) + 1)
ax1.bar(x, [1-s for s in query_sparsities], alpha=0.7, color='steelblue')
ax1.set_xlabel('Query ID', fontsize=12)
ax1.set_ylabel('Activation Rate', fontsize=12)
ax1.set_title('Query Activation Rates', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 0.05)
ax1.grid(True, alpha=0.3)
for i, v in enumerate(query_activated):
    ax1.text(i+1, (1-query_sparsities[i])+0.001, str(v), ha='center', va='bottom', fontsize=9)

# Activated tokens
ax2 = axes[1]
ax2.bar(x, query_activated, alpha=0.7, color='coral')
ax2.set_xlabel('Query ID', fontsize=12)
ax2.set_ylabel('Number of Activated Tokens', fontsize=12)
ax2.set_title('Activated Token Counts', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
for i, v in enumerate(query_activated):
    ax2.text(i+1, v+5, str(v), ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(project_root / 'outputs' / 'korean_query_sparsity.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Chart saved to: outputs/korean_query_sparsity.png")

## 12. Query-Document 유사도 계산

In [ ]:
# 모든 쿼리와 문서 인코딩
print("Encoding all queries and documents...")

def encode_batch(texts: List[str], model: SPLADEDoc, tokenizer: AutoTokenizer) -> torch.Tensor:
    """배치 텍스트를 인코딩합니다."""
    reps = []
    for text in texts:
        rep = encode_text(text, model, tokenizer)
        reps.append(rep)
    return torch.stack(reps)

with torch.no_grad():
    query_reps = encode_batch(korean_queries, model, tokenizer)
    doc_reps = encode_batch(korean_documents, model, tokenizer)

print(f"Query representations: {query_reps.shape}")
print(f"Document representations: {doc_reps.shape}")

# 유사도 행렬 계산 (dot product)
similarity_matrix = torch.mm(query_reps, doc_reps.t()).cpu().numpy()

print(f"\nSimilarity matrix: {similarity_matrix.shape}")
print("\nSimilarity scores (Query × Document):")
print(pd.DataFrame(
    similarity_matrix,
    index=[f"Q{i+1}" for i in range(len(korean_queries))],
    columns=[f"D{i+1}" for i in range(len(korean_documents))],
).round(4))

## 13. 유사도 히트맵 시각화

In [ ]:
# 유사도 히트맵
plt.figure(figsize=(10, 8))

sns.heatmap(
    similarity_matrix,
    annot=True,
    fmt='.3f',
    cmap='YlOrRd',
    xticklabels=[f"Doc {i+1}" for i in range(len(korean_documents))],
    yticklabels=[f"Query {i+1}" for i in range(len(korean_queries))],
    cbar_kws={'label': 'Similarity Score'},
)

plt.title('Query-Document Similarity Matrix\n(Neural Sparse Encoding)', fontsize=14, fontweight='bold')
plt.xlabel('Documents', fontsize=12)
plt.ylabel('Queries', fontsize=12)
plt.tight_layout()
plt.savefig(project_root / 'outputs' / 'similarity_heatmap_korean.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Heatmap saved to: outputs/similarity_heatmap_korean.png")

## 14. Top-K 검색 결과

In [ ]:
# 각 쿼리에 대한 Top-3 문서 검색
print("="*80)
print("TOP-3 RETRIEVAL RESULTS")
print("="*80)

for i, query in enumerate(korean_queries):
    print(f"\n[Query {i+1}] {query}")
    print("-" * 80)
    
    # 상위 3개 문서
    top_k_indices = np.argsort(similarity_matrix[i])[-3:][::-1]
    
    for rank, doc_idx in enumerate(top_k_indices, 1):
        score = similarity_matrix[i][doc_idx]
        doc = korean_documents[doc_idx]
        print(f"\n  Rank {rank} [Score: {score:.4f}] (Doc {doc_idx+1})")
        print(f"  {doc}")

print("\n" + "="*80)

## 15. 결과 요약

In [ ]:
print("="*80)
print("INFERENCE TEST SUMMARY")
print("="*80)

print(f"\n📝 Test Configuration:")
print(f"  Model: {model_name}")
print(f"  Model path: {model_path}")
print(f"  Device: {device}")
print(f"  Vocab size: {vocab_size:,}")

print(f"\n📊 Dataset:")
print(f"  Queries: {len(korean_queries)}")
print(f"  Documents: {len(korean_documents)}")
print(f"  Language: Korean")

print(f"\n🎯 Sparsity Statistics:")
avg_sparsity = np.mean([a['stats']['sparsity'] for a in query_analyses])
avg_activated = np.mean([a['stats']['num_activated'] for a in query_analyses])
print(f"  Average sparsity: {avg_sparsity:.2%}")
print(f"  Average activated tokens: {avg_activated:.1f}")
print(f"  Activation rate: {(1-avg_sparsity)*100:.3f}%")

print(f"\n🔍 Similarity Statistics:")
print(f"  Mean similarity: {similarity_matrix.mean():.4f}")
print(f"  Max similarity: {similarity_matrix.max():.4f}")
print(f"  Min similarity: {similarity_matrix.min():.4f}")
print(f"  Std similarity: {similarity_matrix.std():.4f}")

print(f"\n✅ Inference test completed successfully!")
print(f"\n📁 Generated files:")
print(f"  - outputs/korean_query_sparsity.png")
print(f"  - outputs/similarity_heatmap_korean.png")

print("\n" + "="*80)